In [40]:
import random

# ===============================
# TRAIN / TEST SPLIT (80 / 20)
# ===============================
random.seed(42)

indices = list(range(len(X)))
random.shuffle(indices)

split_idx = int(0.8 * len(indices))

train_idx = indices[:split_idx]
test_idx  = indices[split_idx:]

X_train = [X[i] for i in train_idx]
y_train = [y[i] for i in train_idx]

X_test = [X[i] for i in test_idx]
y_test = [y[i] for i in test_idx]

print("Train size:", len(X_train))
print("Test size :", len(X_test))


Train size: 569
Test size : 143


In [41]:
# ===============================
# HELPER FUNCTIONS
# ===============================

def find_mean(dataset):
    return sum(dataset) / len(dataset)

def find_standard_deviation(dataset, mean):
    return (sum((x - mean) ** 2 for x in dataset) / len(dataset)) ** 0.5


# ===============================
# COMPUTE MEANS & STDS (TRAIN ONLY)
# ===============================

feature_means = []
feature_stds = []

for feature_idx in range(n_features):
    column_values = [row[feature_idx] for row in X_train]

    mean = find_mean(column_values)
    std = find_standard_deviation(column_values, mean)

    if std == 0:
        std = 1.0

    feature_means.append(mean)
    feature_stds.append(std)

print("Means:", feature_means)
print("Stds :", feature_stds)


Means: [-0.01819094427569709, 6.670096280728936e-05, -0.030146853412043317, -0.0254946749221174, -0.00028917637070838465, 0.01784400504652037, -0.005541074762952938]
Stds : [1.0016074081481439, 1.0000188864182753, 0.9876400712252256, 1.0000286268431353, 1.0156419357139692, 1.0491042361967042, 0.994998472526364]


In [ ]:
import math
import random

# ===============================
# HYPERPARAMETERS
# ===============================
learning_rate = 0.005
lambda_ = 0.0001
epochs = 15000

# ===============================
# DATA SIZES
# ===============================
n_samples = len(X_train)
n_features = len(X_train[0])

# ===============================
# INITIALIZE PARAMETERS
# ===============================
weight = [random.uniform(-0.01, 0.01) for _ in range(n_features)]
b = 0.0


# ===============================
# MODEL FUNCTIONS
# ===============================
def sigmoid(z):
    if z >= 0:
        return 1 / (1 + math.exp(-z))
    else:
        ez = math.exp(z)
        return ez / (1 + ez)


def find_weighted_sum_for_row(row):
    return sum(weight[i] * row[i] for i in range(n_features)) + b


# ===============================
# TRAINING
# ===============================
def train_one_epoch():
    global b

    preds = [
        sigmoid(find_weighted_sum_for_row(X_train[j]))
        for j in range(n_samples)
    ]

    # update weights
    for i in range(n_features):
        grad = sum(
            (preds[j] - y_train[j]) * X_train[j][i]
            for j in range(n_samples)
        ) / n_samples

        # L2 regularization (no bias)
        grad += lambda_ * weight[i]

        weight[i] -= learning_rate * grad

    # update bias
    b_grad = sum(preds[j] - y_train[j] for j in range(n_samples)) / n_samples
    b -= learning_rate * b_grad


def train_model(epochs):
    for epoch in range(epochs):
        train_one_epoch()
        if epoch % 1000 == 0:
            print(f"Epoch {epoch}")


# ===============================
# TRAIN
# ===============================
train_model(epochs)

print("\nTrained weights:", weight)
print("Trained bias:", b)


# ===============================
# TEST ON TEST SET
# ===============================
print("\nTest predictions (first 5):")
for i in range(5):
    p = sigmoid(find_weighted_sum_for_row(X_test[i]))
    print(f"Sample {i}: probability={p:.4f}, predicted={1 if p>=0.5 else 0}, actual={y_test[i]}")


Epoch 0
Epoch 1000
Epoch 2000
Epoch 3000
Epoch 4000
Epoch 5000
Epoch 6000
Epoch 7000
Epoch 8000
Epoch 9000
Epoch 10000
Epoch 11000
Epoch 12000
Epoch 13000
Epoch 14000

Trained weights: [-1.039574018158522, 1.2640157063505881, -0.6227957188980372, -0.29810272188808645, -0.03452107106915563, 0.10383542709986539, 0.03590202166911438]
Trained bias: -0.5354805217899608

Test predictions (first 5):
Sample 0: probability=0.6430, predicted=1, actual=1
Sample 1: probability=0.1340, predicted=0, actual=0
Sample 2: probability=0.9111, predicted=1, actual=1
Sample 3: probability=0.4471, predicted=0, actual=0
Sample 4: probability=0.1391, predicted=0, actual=1
